In [1]:
import pandas as pd
from collections import Counter
import torch
from sklearn.model_selection import train_test_split
from networkx.readwrite.json_graph import adjacency
import random, math, time, os
import torch.nn.functional as F
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader



In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
dataset_pd = pd.read_csv('D:\\VideoRecSystem\\MicroLens\\DataSet\\MicroLens-50k_pairs.csv')
# dataset_pd = pd.read_csv('MicroLens-50k_pairs.csv')

In [4]:
dataset_pd.head(10)

,user,item,timestamp
0,36121,9580,1583378629552
1,26572,9580,1583436719018
2,37550,9580,1584412681021
3,14601,9580,1584848802432
4,15061,9580,1585388171106
5,6364,9580,1585390736041
6,3542,9580,1585404918503
7,21038,9580,1590144594477
8,12538,14631,1634867362929
9,47592,14631,1634872254913


In [5]:
dataset_pd.count

<bound method DataFrame.count of          user   item      timestamp
0       36121   9580  1583378629552
1       26572   9580  1583436719018
2       37550   9580  1584412681021
3       14601   9580  1584848802432
4       15061   9580  1585388171106
...       ...    ...            ...
359703  48702   1363  1662984066647
359704  27203   7291  1662984082974
359705  29261  19649  1662984103874
359706  28341  19188  1662984123833
359707  38967   7254  1662984132429

[359708 rows x 3 columns]>

In [6]:
user_counts = dataset_pd['user'].value_counts()
item_counts = dataset_pd['item'].value_counts()
# valid_users = user_counts[user_counts > 3].index
# valid_items = item_counts[item_counts > 3].index
# filtered_df = dataset_pd[dataset_pd['user'].isin(valid_users) & dataset_pd['item'].isin(valid_items)]
# filtered_df.count

In [7]:
# order by user,timestamp 
filtered_df = dataset_pd.sort_values(by=["user", "timestamp"])


In [8]:
def split(df, user_col='user', item_col='item', time_col='timestamp'):

    df = df.sort_values(by=[user_col, time_col])  # 按用户时间排序

    # 获取每个用户的最后一条记录作为 test
    test_df = df.groupby(user_col).tail(1)
    train_df = df.drop(index=test_df.index)

    # 过滤 test 中那些 user/item 不在 train 中的
    train_users = set(train_df[user_col])
    train_items = set(train_df[item_col])

    test_df = test_df[
        test_df[user_col].isin(train_users) &
        test_df[item_col].isin(train_items)
    ]

    return train_df.reset_index(drop=True), test_df.reset_index(drop=True)

In [9]:

train_df, test_df = split(filtered_df,user_col='user', item_col='item', time_col='timestamp')

print(f"Train size: {len(train_df)}")
print(f"Test size: {len(test_df)}")


Train size: 309708
Test size: 49424


In [10]:
# maintain a map from new id to old id, new id for constructing matrix
user2id = {u: i for i, u in enumerate(filtered_df['user'].unique())}
item2id = {i: j for j, i in enumerate(filtered_df['item'].unique())}

# apply to train_df and test_df
train_df['user_id'] = train_df['user'].map(user2id)
train_df['item_id'] = train_df['item'].map(item2id)
test_df['user_id'] = test_df['user'].map(user2id)
test_df['item_id'] = test_df['item'].map(item2id)

num_users = len(user2id)
num_items = len(item2id)


In [11]:
# ---------- 超参数 ----------
MAX_SEQ_LEN   = 20          # 序列长度
EMBED_DIM     = 64          # item / user embedding 维度
N_HEADS       = 2           # Multi-Head Attention 头数
N_LAYERS      = 2           # Transformer block 层数
DROPOUT       = 0.2
NEG_SAMPLE    = 5
BATCH_SIZE    = 512
EPOCHS        = 10
LR            = 1e-3
SEED          = 42
# ----------------------------

torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

# ---------- 常量（来自你已有变量） ----------
PAD_IDX = num_items          # 专用 padding id
N_ITEMS = num_items + 1      # Embedding 行数（含 PAD）
ALL_ITEM_IDS = np.arange(num_items, dtype=np.int64)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# -------------------------------------------

# ======== Dataset (同前) ======== #
# ======== Dataset：与前一致 ======== #
class SASRecBPRDataset(Dataset):
    def __init__(self, df, max_len=MAX_SEQ_LEN, pad_idx=PAD_IDX, n_neg=NEG_SAMPLE):
        self.max_len, self.pad_idx, self.n_neg = max_len, pad_idx, n_neg
        self.inputs, self.targets = [], []

        for _, hist in df.groupby('user_id'):
            seq = hist['item_id'].tolist()
            for i in range(1, len(seq)):
                s = seq[max(0, i-max_len): i]
                s = [pad_idx]*(max_len-len(s)) + s
                self.inputs.append(s)
                self.targets.append(seq[i])

        self.inputs  = np.asarray(self.inputs,  dtype=np.int64)
        self.targets = np.asarray(self.targets, dtype=np.int64)

    def __len__(self): return len(self.targets)

    def _neg(self, pos):
        negs = np.random.choice(ALL_ITEM_IDS, size=self.n_neg, replace=False)
        while (negs == pos).any():
            dup = negs == pos
            negs[dup] = np.random.choice(ALL_ITEM_IDS, size=dup.sum(), replace=False)
        return negs

    def __getitem__(self, idx):
        hist = torch.tensor(self.inputs[idx], dtype=torch.long)
        pos  = torch.tensor(self.targets[idx], dtype=torch.long)
        negs = torch.tensor(self._neg(self.targets[idx]), dtype=torch.long)
        return hist, pos, negs



In [12]:
# ======== SASRec 模型 ======== #
class SASRec(nn.Module):
    def __init__(self,
                 n_items=N_ITEMS,
                 dim=EMBED_DIM,
                 n_heads=N_HEADS,
                 n_layers=N_LAYERS,
                 max_len=MAX_SEQ_LEN,
                 pad_idx=PAD_IDX,
                 dropout=DROPOUT):
        super().__init__()
        self.item_emb = nn.Embedding(n_items, dim, padding_idx=pad_idx)
        self.pos_emb  = nn.Embedding(max_len, dim)
        self.dropout  = nn.Dropout(dropout)

        enc_layer = nn.TransformerEncoderLayer(
            d_model=dim,
            nhead=n_heads,
            dim_feedforward=dim*4,
            dropout=dropout,
            batch_first=True,
            activation='gelu')
        self.encoder = nn.TransformerEncoder(enc_layer, num_layers=n_layers)

        # 为 causal mask 预生成上三角矩阵
        self.register_buffer(
            "mask", torch.triu(torch.ones(max_len, max_len), diagonal=1).bool())

    def forward(self, seq):                       # seq : (B,T)
        B, T = seq.size()
        item_e = self.item_emb(seq)               # (B,T,D)
        pos_ids = torch.arange(T, device=seq.device).unsqueeze(0).expand(B, -1)
        x = item_e + self.pos_emb(pos_ids)        # 加位置编码
        x = self.dropout(x)

        # causal attention mask
        causal_mask = self.mask[:T, :T]
        x = self.encoder(x, src_key_padding_mask=(seq == PAD_IDX), mask=causal_mask)
        h = x[:, -1, :]                           # 取最后位置向量 (B,D)
        return h

    def score(self, h, items):                    # items : (B,) 或 (B,k)
        item_e = self.item_emb(items)
        return (h.unsqueeze(-2) * item_e).sum(-1)


In [13]:
# ======== BPR 损失 ======== #
def bpr_loss(pos_s, neg_s):
    return -torch.log(torch.sigmoid(pos_s.unsqueeze(-1) - neg_s) + 1e-8).mean()



In [14]:
# ======== 训练函数 ======== #
def train_sasrec_bpr(train_df, test_df=None):
    ds = SASRecBPRDataset(train_df)
    loader = DataLoader(ds, batch_size=BATCH_SIZE, shuffle=True,
                        num_workers=0, pin_memory=True)

    model = SASRec().to(DEVICE)
    optim = torch.optim.Adam(model.parameters(), lr=LR)

    for ep in range(1, EPOCHS+1):
        model.train()
        tot_loss, t0 = 0.0, time.time()
        for step, (hist, pos, neg) in enumerate(loader, 1):
            hist, pos, neg = hist.to(DEVICE), pos.to(DEVICE), neg.to(DEVICE)
            h = model(hist)
            pos_s = model.score(h, pos)
            neg_s = model.score(h, neg)
            loss  = bpr_loss(pos_s, neg_s)

            optim.zero_grad()
            loss.backward()
            optim.step()

            tot_loss += loss.item()*hist.size(0)
            if step % 100 == 0 or step == 1:
                print(f"[Ep {ep}] step {step}/{len(loader)} | loss {loss.item():.4f}", flush=True)

        print(f"Ep {ep} done | avg loss {tot_loss/len(ds):.4f} | {time.time()-t0:.1f}s\n", flush=True)
    return model

In [15]:
# ======== 构建 (user → 历史张量) 缓存 ======== #
def build_hist_cache(df):
    cache = {}
    for uid, items in df.groupby('user_id')['item_id']:
        seq = items.tolist()[-MAX_SEQ_LEN:]
        seq = [PAD_IDX]*(MAX_SEQ_LEN - len(seq)) + seq
        cache[uid] = torch.tensor(seq, dtype=torch.long).unsqueeze(0)
    return cache

In [25]:
num_users

50000

In [26]:

def evaluate_ranking(
        test_df,              # DataFrame, 必含 user_id / item_id
        train_df,             # DataFrame, 用来构建用户→已交互物品集合
        score_fn,             # callable(users_tensor, items_tensor) → np.array
        num_items,            # 物品总数
        k=10,                 # Hit@K / NDCG@K
        num_neg=10000,          # 每个正样本采多少负样本
        user_col='user_id',
        item_col='item_id',
        seed=42
    ):
    """
    不依赖具体模型，只要提供 score_fn 就能评估。
    score_fn: 接收 (user_tensor, item_tensor) 并返回同长度的 Numpy 分数向量。
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

    # 用户历史，用于采负样本 & 过滤
    train_user_dict = (
        train_df.groupby(user_col)[item_col].apply(set).to_dict()
    )

    hits, ndcgs = [], []

    for _, row in test_df.iterrows():
        u = int(row[user_col])
        pos_item = int(row[item_col])

        # ---------- 负采样 ----------
        neg_items = set()
        while len(neg_items) < num_neg:
            neg = random.randint(0, num_items - 1)
            if neg not in train_user_dict.get(u, set()) and neg != pos_item:
                neg_items.add(neg)

        item_candidates = list(neg_items) + [pos_item]

        # ---------- 评分 ----------
        users_t  = torch.LongTensor([u] * len(item_candidates))
        items_t  = torch.LongTensor(item_candidates)
        scores   = score_fn(users_t, items_t)        # ← 只依赖 score_fn
        rank_idx = np.argsort(scores)[::-1]          # 降序
        ranked_items = [item_candidates[i] for i in rank_idx]

        # ---------- 指标 ----------
        if pos_item in ranked_items[:k]:
            hits.append(1)
            rank_pos = ranked_items.index(pos_item)
            ndcgs.append(1 / np.log2(rank_pos + 2))
        else:
            hits.append(0)
            ndcgs.append(0)

    hit_rate = float(np.mean(hits))
    ndcg     = float(np.mean(ndcgs))
    return hit_rate, ndcg

In [17]:
    # ------------------ 训练 ------------------
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model  = train_sasrec_bpr(train_df, test_df)

[Ep 1] step 1/508 | loss 4.4320
[Ep 1] step 100/508 | loss 3.3622
[Ep 1] step 200/508 | loss 2.3144
[Ep 1] step 300/508 | loss 1.7428
[Ep 1] step 400/508 | loss 1.2451
[Ep 1] step 500/508 | loss 0.9118
Ep 1 done | avg loss 2.1889 | 197.6s

[Ep 2] step 1/508 | loss 0.9144
[Ep 2] step 100/508 | loss 0.7561
[Ep 2] step 200/508 | loss 0.7230
[Ep 2] step 300/508 | loss 0.6795
[Ep 2] step 400/508 | loss 0.6865
[Ep 2] step 500/508 | loss 0.6798
Ep 2 done | avg loss 0.7240 | 190.1s

[Ep 3] step 1/508 | loss 0.6465
[Ep 3] step 100/508 | loss 0.6473
[Ep 3] step 200/508 | loss 0.6133
[Ep 3] step 300/508 | loss 0.6335
[Ep 3] step 400/508 | loss 0.6117
[Ep 3] step 500/508 | loss 0.5873
Ep 3 done | avg loss 0.6296 | 227.0s

[Ep 4] step 1/508 | loss 0.6314
[Ep 4] step 100/508 | loss 0.6080
[Ep 4] step 200/508 | loss 0.5872
[Ep 4] step 300/508 | loss 0.5316
[Ep 4] step 400/508 | loss 0.5851
[Ep 4] step 500/508 | loss 0.5617
Ep 4 done | avg loss 0.5716 | 205.8s

[Ep 5] step 1/508 | loss 0.5435
[Ep 5] s

In [27]:
def make_popularity_score_fn(train_df, item_col='item_id'):
    item_cnt = Counter(train_df[item_col])
    default_score = min(item_cnt.values()) - 1  # 给没出现过的物品一个更低分
    def _score_fn(users_t, items_t):
        return np.array([item_cnt.get(int(i), default_score) for i in items_t])
    return _score_fn

In [28]:
def random_score_fn(users_t, items_t):
    # 随机给每个 items_t 一个分数；users_t 不使用，但必须接收
    return np.random.rand(len(items_t))

In [29]:
# ======== make_score_fn：供 evaluate_ranking 用 ======== #
def make_score_fn(model, hist_cache):
    model.eval()
    item_table = model.item_emb.weight.data           # 含 PAD
    @torch.no_grad()
    def score_fn(u_t, i_t):
        hist_batch = torch.cat([hist_cache[int(u)] for u in u_t], dim=0).to(DEVICE)
        h = model(hist_batch)
        item_e = item_table[i_t.to(DEVICE)]
        return (h * item_e).sum(-1).cpu().numpy()
    return score_fn

In [30]:

# ----------------SASRec （或其他模型）------------
score_fn_SASRec = make_score_fn(model,hist_cache=build_hist_cache(train_df))
hit_SASRec, ndcg_SASRec = evaluate_ranking(
    test_df, train_df, score_fn_SASRec,
    num_items=num_items, k=10
)
print(f"SASRec   Hit@10={hit_SASRec:.4f}  NDCG@10={ndcg_SASRec:.4f}")

# ---------------- baseline：Popular ----------------
pop_score_fn  = make_popularity_score_fn(train_df)
hit_pop, ndcg_pop = evaluate_ranking(
    test_df, train_df, pop_score_fn,
    num_items=num_items, k=10
)
print(f"Popular  Hit@10={hit_pop:.4f}  NDCG@10={ndcg_pop:.4f}")

# ---------------- baseline：Random -----------------
hit_rand, ndcg_rand = evaluate_ranking(
    test_df, train_df, random_score_fn,
    num_items=num_items, k=10
)
print(f"Random   Hit@10={hit_rand:.4f}  NDCG@10={ndcg_rand:.4f}")



SASRec   Hit@10=0.9847  NDCG@10=0.9845


KeyboardInterrupt: 